# Preamble

In [3]:
import $ivy.`org.typelevel::cats-core:2.13.0`
import $ivy.`org.typelevel::cats-free:2.13.0`

import $ivy.$                                

import $ivy.$                                


In [4]:
import cats.*, cats.syntax.all.*
import cats.free.* 
import Tuple.*

import cats.*, cats.syntax.all.*

import cats.free.* 

import Tuple.*


#### Rose trees

In [5]:
case class Tree[+A](a: String, children: List[Tree[?]])

object Tree: 

    given [A]: Show[Tree[A]] with 

        def toString(t: Tree[?])(prefix: String, bar: String, sep: String): String = 
            prefix + sep + t.a.toString + 
            (t.children match 
                case List() => "" 
                case head :: tail => 
                    "\n" + 
                    (if tail.isEmpty then ""
                    else tail.map(toString(_)(prefix + bar + "   ", "|", "├──"))
                            .mkString("\n") + "\n") +
                    toString(head)(prefix + bar + "   ", "", "└──")       
            )

        def show(t: Tree[A]): String = 
            toString(t)("", "", "")

defined class Tree
defined object Tree

#### Higher-order features

In [6]:
infix type ~>[F[_], G[_]] = [A] => F[A] => G[A]

defined type ~>

In [7]:
infix type ~~>[F[_[_]] <: [A] =>> Any, G[_[_]] <: [A] =>> Any] = [T[_], A] => F[T][A] => G[T][A]

defined type ~~>

In [8]:
import scala.language.implicitConversions

import scala.language.implicitConversions


In [8]:
/*implicit def nat[S[_], T[_]](f: [A] => S[A] => T[A]): S ~> T =
    new (S ~> T): 
        def apply[A](x: S[A]): T[A] = f(x)*/

In [9]:
trait HFunctor[F[_[_]] <: [t] =>> Any]: 
    def hfmap[G[_], H[_]](nat: G ~> H): F[G] ~> F[H]
   
object HFunctor: 

    def apply[F[_[_]] <: [t] =>> Any](using H: HFunctor[F]): HFunctor[F] = H

defined trait HFunctor
defined object HFunctor

In [10]:
infix type Compose[G[_], F[_]] = [t] =>> G[F[t]]

defined type Compose

In [11]:
infix type HCompose[G[_], F[_[_]] <: [t] =>> Any] = [t[_]] =>> [a] =>> G[F[t][a]]

given HCompose_Functor[G[_]: Functor, F[_[_]] <: [t] =>> Any : HFunctor]: HFunctor[G HCompose F] with 
    def hfmap[H[_], I[_]](hToi: H ~> I): HCompose[G, F][H] ~> HCompose[G, F][I] = 
        [A] => (gh: G[F[H][A]]) =>
            val fnat: F[H] ~> F[I] = HFunctor[F].hfmap(hToi)
            gh map fnat[A]

defined type HCompose
defined class HCompose_Functor

In [12]:
trait HTraverse[F[_[_]] <: [t] =>> Any]: 
    def traverse[H[_], I[_]: Applicative, J[_]](f: H ~> (I Compose J)): F[H] ~> (I HCompose F)[J]

object HTraverse: 

    def apply[F[_[_]] <: [t] =>> Any](using H: HTraverse[F]): HTraverse[F] = H

defined trait HTraverse
defined object HTraverse

In [13]:
type HMu[F[_[_]] <: [t] =>> Any] = [A] =>> () => HMuA[F, A]

case class HMuA[F[_[_]] <: [t] =>> Any, A](out: F[HMu[F]][A])

object HMuA: 

    def in[F[_[_]] <: [t] =>> Any]: ([t] =>> () => F[HMu[F]][t]) ~> HMu[F] = 
        [A] => (f: () => F[HMu[F]][A]) => () => HMuA[F, A](f())

    def map[F[_[_]] <: [t] =>> Any : HFunctor, G[_[_]] <: [A] =>> Any](nnat: F ~~> G): HMu[F] ~> HMu[G] = 
        [A] => (h: HMu[F][A]) => 
            val nat: F[HMu[F]] ~> F[HMu[G]] = HFunctor[F].hfmap(map(nnat))
            () => HMuA[G, A](nnat[HMu[G], A](nat[A](h().out)))

    def map_atDepth[F[_[_]] <: [t] =>> Any : HFunctor, G[_[_]] <: [A] =>> Any](nnat: Int => (F ~~> G)): HMu[F] ~> HMu[G] = 
        def go(level: Int): HMu[F] ~> HMu[G] = 
            [A] => (h: HMu[F][A]) => 
                val nat: F[HMu[F]] ~> F[HMu[G]] = HFunctor[F].hfmap(go(level+1))
                () => HMuA[G, A](nnat(level).apply[HMu[G], A](nat[A](h().out)))

        go(0)

    def truncate[F[_[_]] <: [t] =>> Any : HFunctor](depth: Int): HMu[F] ~> HMu[Option HCompose F] = 
        map_atDepth[F, Option HCompose F](
            level => [T[_], A] => (f: F[T][A]) => 
                if level == depth then None else Some(f)
        )

    def fold[F[_[_]] <: [t] =>> Any : HFunctor, H[_]](alg: F[H] ~> H): HMu[F] ~> H = 
        [A] => (hf: () => HMuA[F, A]) => 
            val nat: F[HMu[F]] ~> F[H] = HFunctor[F].hfmap(fold(alg))
            alg[A](nat[A](hf().out))

    def unfold[H[_], F[_[_]] <: [t] =>> Any : HFunctor](coalg: H ~> F[H]): H ~> HMu[F] =
        [A] => (h: H[A]) =>
            val nat: F[H] ~> F[HMu[F]] = HFunctor[F].hfmap(unfold(coalg))
            () => HMuA[F, A](nat[A](coalg[A](h)))

    def sequenceF_Rec[G[_]: Monad, H[_[_]] <: [t] =>> Any : HTraverse](rec: HMu[G HCompose H] ~> (G Compose HMu[H])):
            HMu[G HCompose H] ~> (G Compose HMu[H]) =
        [A] => (ss: HMu[G HCompose H][A]) =>
            Monad[G].flatMap(ss().out): (h: H[HMu[G HCompose H]][A]) => 
                val nat: H[HMu[G HCompose H]] ~> (G HCompose H)[HMu[H]] = 
                    HTraverse[H].traverse[HMu[G HCompose H], G, HMu[H]]([B] => hmu => rec(hmu))
                Functor[G].map(nat[A](h))(h => in[H][A](() => h))

    def sequenceF[G[_]: Monad, H[_[_]] <: [t] =>> Any : HTraverse]: HMu[G HCompose H] ~> (G Compose HMu[H]) =
        sequenceF_Rec(sequenceF[G, H])

    def sequenceF_Until[G[_]: Monad: Alternative, H[_[_]] <: [t] =>> Any : HTraverse](depth: Int): 
          HMu[G HCompose H] ~> (G Compose HMu[H]) = 
        if depth == 0 then [A] => _ => Alternative[G].empty
        else sequenceF_Rec(sequenceF_Until[G, H](depth-1))

    def show[F[_[_]] <: [t] =>> Any : HFunctor](depth: Int)(using alg: F[Tree] ~> Tree): HMu[F] ~> ([_] =>> String) = 
        given HFunctor[Option HCompose F] = HCompose_Functor[Option, F]

        val optAlg: (Option HCompose F)[Tree] ~> Tree = [A] => (of: Option[F[Tree][A]]) => 
            of match 
                case None => Tree("...", Nil)
                case Some(f) => alg[A](f)
                    
        [A] => (h: HMu[F][A]) => 
            val truncated: HMu[Option HCompose F][A] = truncate[F](depth)(h)
            val tree: Tree[A] = fold[Option HCompose F, Tree](optAlg)(truncated)
            tree.show


defined type HMu
defined class HMuA
defined object HMuA

In [14]:
type HFix[F[_[_]] <: [t] =>> Any] = [A] =>> HFixA[F, A]
          
case class HFixA[F[_[_]] <: [t] =>> Any, A](out: F[HFix[F]][A])

object HFixA: 

    def from[F[_[_]] <: [t] =>> Any : HFunctor]: HMu[F] ~> HFix[F] =
        [A] => (mu: HMu[F][A]) => 
            val nat: F[HMu[F]] ~> F[HFix[F]] = HFunctor[F].hfmap(from[F])
            HFixA[F, A](nat[A](mu().out))

    def fold[F[_[_]] <: [t] =>> Any : HFunctor, H[_]](alg: F[H] ~> H): HFix[F] ~> H =
        [A] => (hf: HFix[F][A]) =>
            val nat: F[HFix[F]] ~> F[H] = HFunctor[F].hfmap(fold(alg))
            alg[A](nat[A](hf.out))


defined type HFix
defined class HFixA
defined object HFixA

# Sequents & types

In [15]:
import scala.reflect.{ClassTag, classTag}

import scala.reflect.{ClassTag, classTag}


In [16]:
enum Type[T]: 
    case atom[T](s: String) extends Type[T]
    case implies[T1, T2](t1: Type[T1], t2: Type[T2]) extends Type[T1 => T2]

object Type:

    def apply[T](using T: Type[T]): Type[T] = T

    given Type_Implies[T1, T2](using t1: Type[T1], t2: Type[T2]): Type[T1 => T2] = Type.implies(t1, t2)

    given Type_Atom[T: ClassTag]: Type[T] = 
        val name = classTag[T].runtimeClass.getName()
        atom[T]((""".*\$(.*)""".r).findFirstMatchIn(name).fold(name)(_.group(1)))

    def show(t: Type[?]): String = 
        t match 
            case atom(s) => s
            case implies(t1, t2) => "(" + show(t1) + " => " + show(t2) + ")"

defined class Type
defined object Type

In [17]:
type Env[T <: Tuple] <: Tuple = T match 
    case EmptyTuple => EmptyTuple
    case h *: t => Type[h] *: Env[t]

object Env: 
    def apply[T <: Tuple](using E: Env[T]): Env[T] = E

    given Env[EmptyTuple] = EmptyTuple
    given [H: Type, T <: Tuple: Env]: Env[H *: T] = Type[H] *: summon[Env[T]]

    def show(env: Tuple): String = 
        (env: @unchecked) match 
            case EmptyTuple => ""
            case (h: Type[?]) *: EmptyTuple => Type.show(h)
            case (h: Type[?]) *: tail  => Type.show(h) + ", " + show(tail)

defined type Env
defined object Env

In [18]:
import Env.given

import Env.given


In [19]:
case class Sequent[E <: Tuple, A](env: Env[E], con: Type[A])

object Sequent: 
    def apply[E <: Tuple, A](using S: Sequent[E, A]): Sequent[E, A] = S 

    given [E <: Tuple: Env, A: Type]: Sequent[E, A] = 
        Sequent(Env[E], Type[A])

    def show(s: Sequent[?, ?]): String = 
        Env.show(s.env) + " ⊢ " + Type.show(s.con)

defined class Sequent
defined object Sequent

#### Tuples

In [20]:
trait IndexOf[T <: Tuple, A]: 
    val i: Int
    val eq: Elem[T, i.type] =:= A

object IndexOf: 
    def apply[T <: Tuple, A](j: Int)(using Elem[T, j.type] =:= A): IndexOf[T, A] = 
        new IndexOf[T, A]: 
            val i: j.type = j
            val eq = summon

defined trait IndexOf
defined object IndexOf

In [21]:
def indexOf[T <: Tuple, A](a: A)(t: T): Option[IndexOf[T, A]] = 
    def go(out: Int, t: Tuple): Option[IndexOf[T, A]] = 
        t match 
            case EmptyTuple => None
            case `a` *: _ => Some(IndexOf[T, A](out)(using <:<.refl[Elem[T, out.type]].asInstanceOf[Elem[T, out.type] =:= A]))
            case _ *: tail => go(out+1, tail)

    go(0, t)

defined function indexOf

In [22]:
trait IndexOfImpl[T <: Tuple]: 
    val i: Int
    type a: Type
    type b: Type
    val eq: Elem[T, i.type] =:= (a => b)
    
object IndexOfImpl: 
    def apply[T <: Tuple, A: Type, B: Type](j: Int)(using Elem[T, j.type] =:= (A => B)): IndexOfImpl[T] = 
        new IndexOfImpl[T]: 
            val i: j.type = j
            type a = A
            type b = B
            val eq = summon

defined trait IndexOfImpl
defined object IndexOfImpl

In [23]:
def indexOfImpl[T <: Tuple](t: T): LazyList[IndexOfImpl[T]] = 
    def go(i: Int, out: LazyList[IndexOfImpl[T]], t: Tuple): LazyList[IndexOfImpl[T]] = 
        t match
            case EmptyTuple => out.reverse
            case Type.implies(ta: Type[a], tb: Type[b]) *: tail => 
                go(i+1, IndexOfImpl(i)(using ta, tb, <:<.refl[Elem[T, i.type]].asInstanceOf[Elem[T, i.type] =:= (a => b)]) +: out, tail)
            case _ *: tail => go(i+1, out, tail)

    go(0, LazyList.empty, t)

defined function indexOfImpl

# Natural Deduction/Lambda Calculus Minimal Propositional Logic

### Tagless-final

In [24]:
//trait Lambda[P[_ <: Sequent[_, _]]]:
trait ND[P[_]]:
    def v[Γ <: Tuple, A](i: Int)(using Elem[Γ, i.type] =:= A): P[Sequent[Γ, A]]
    def abs[Γ <: Tuple, A, B](t: P[Sequent[A *: Γ, B]]): P[Sequent[Γ, A => B]]
    def app[Γ <: Tuple, A, B](f: P[Sequent[Γ, A => B]])(t1: P[Sequent[Γ, A]]): P[Sequent[Γ, B]]
    def substz[Γ <: Tuple, A, B](t1: P[Sequent[Γ, A]])(t2: P[Sequent[A *: Γ, B]]): P[Sequent[Γ, B]]

object ND:

    type Std[S] <: Function1[?, ?] = S match 
        case Sequent[e, a] => (e => a)
                
    given Std_ND: ND[Std] with 
        def v[E <: Tuple, A](i: Int)(using Elem[E, i.type] =:= A): E => A = _(i)
        def abs[E <: Tuple, T1, T2](t: (T1 *: E) => T2): E => T1 => T2 = e => t1 => t(t1 *: e)
        def app[E <: Tuple, T1, T2](f: E => T1 => T2)(t1: E => T1): E => T2 = e => f(e)(t1(e))
        def substz[E <: Tuple, T1, T2](t1: E => T1)(t2: (T1 *: E) => T2): E => T2 = e => t2(t1(e) *: e)

    given Show: ND[[_] =>> String] with 
        def v[E <: Tuple, A](i: Int)(using Elem[E, i.type] =:= A): String = s"v$i"
        def abs[E <: Tuple, T1, T2](t: String): String = s"λ($t)"
        def app[E <: Tuple, T1, T2](f: String)(t1: String): String = s"$f($t1)"
        def substz[E <: Tuple, T1, T2](t1: String)(t2: String): String = s"$t2[$t1/v0]" // t2.replace("v0", t1)


defined trait ND
defined object ND

In [25]:
def t1[P[_]](using L: ND[P]): P[Sequent[EmptyTuple, Int => Int => Int => Int]] = 
    L.abs(L.abs(L.abs(L.v(0))))

t1[ND.Std](EmptyTuple)(2)(1)(0)
t1[[_] =>> String]

defined function t1
res25_1: Int = 0
res25_2: String = "λ(λ(λ(v0)))"

# LJ Minimal Propositional Logic

### Tagles-final

In [26]:
object TF: 

    trait LJ[T[_]]: 

        def ax[Γ <: Tuple, A](i: Int)(using Elem[Γ, i.type] =:= A): T[Sequent[Γ, A]]

        def `l=>`[Γ <: Tuple, A, B, C](
            i: Int, 
            t1: T[Sequent[Γ, A]], 
            t2: T[Sequent[B *: Γ, C]])(using Elem[Γ, i.type] =:= (A => B)): 
                T[Sequent[Γ, C]]

        def `r=>`[Γ <: Tuple, A, B](t: T[Sequent[A *: Γ, B]]): T[Sequent[Γ, A => B]]

    object LJ: 

        given LJ_ND[T[_]](using L: ND[T]): LJ[T] with 

            def ax[Γ <: Tuple, A](i: Int)(using Elem[Γ, i.type] =:= A): T[Sequent[Γ, A]] = 
                L.v(i)

            def `l=>`[Γ <: Tuple, A, B, C](
                    i: Int,
                    t1: T[Sequent[Γ, A]],
                    t2: T[Sequent[B *: Γ, C]])(using eq: Elem[Γ, i.type] =:= (A => B)):
                        T[Sequent[Γ, C]] =
                L.substz(L.app(L.v(i))(t1))(t2)

            def `r=>`[Γ <: Tuple, A, B](t: T[Sequent[A *: Γ, B]]): T[Sequent[Γ, A => B]] = 
                L.abs(t)

defined object TF

In [27]:
def proof1[T[_], A, B](using LJ: TF.LJ[T]): T[Sequent[EmptyTuple, A => B => A]] = 
    LJ.`r=>`(
        LJ.`r=>`(
            LJ.ax(1): T[Sequent[B *: A *: EmptyTuple, A]]
        ): T[Sequent[A *: EmptyTuple, B => A]]
    ): T[Sequent[EmptyTuple, A => B => A]]

defined function proof1

In [28]:
def proof2[T[_], A, B, C](using LJ: TF.LJ[T]): 
        T[Sequent[EmptyTuple, (A => B => C) => (A => B) => A => C]] = 
    LJ.`r=>`(
        LJ.`r=>`(
            LJ.`r=>`(
                LJ.`l=>`(1,
                    LJ.ax(0): T[Sequent[A *: (A => B) *: (A => B => C) *: EmptyTuple, A]], 
                    LJ.`l=>`(3,
                        LJ.ax(1),
                        LJ.`l=>`(0,
                            LJ.ax(1),
                            LJ.ax(0)
                        ): T[Sequent[(B => C) *: B *: A *: (A => B) *: (A => B => C) *: EmptyTuple, C]]
                    ): T[Sequent[B *: A *: (A => B) *: (A => B => C) *: EmptyTuple, C]]
                ): T[Sequent[A *: (A => B) *: (A => B => C) *: EmptyTuple, C]]
            ): T[Sequent[(A => B) *: (A => B => C) *: EmptyTuple, A => C]]
        ): T[Sequent[(A => B => C) *: EmptyTuple, (A => B) => A => C]]
    ): T[Sequent[EmptyTuple, (A => B => C) => (A => B) => A => C]]

defined function proof2

In [29]:
proof1[ND.Std, Int, String](EmptyTuple)(1)("")

res29: Int = 1

In [30]:
proof2[ND.Std, String, Int, Char](EmptyTuple)(
    s => i => (s.toInt + i).toChar)(
    s => 1)(
    "100")

res30: Char = 'e'

### F-algebras

In [31]:
type LJ[A] = HMu[LJ_C][A]

type LJ_C[T[_]] = [A] =>> LJ_F[T, A]

sealed abstract class LJ_F[T[_], S](using S): 
    val tpe: S = summon

object LJ_F: 

    case class ax[T[_], G <: Tuple: Env, A: Type](i: Int)(using Elem[G, i.type] =:= A) 
    extends LJ_F[T, Sequent[G, A]]: 
        val ev: Elem[G, i.type] =:= A = summon

    case class `l=>`[T[_], G <: Tuple: Env, A: Type, B: Type, C: Type](
        i: Int,
        t1: T[Sequent[G, A]], 
        t2: T[Sequent[B *: G, C]] 
    )(using Elem[G, i.type] =:= (A => B)) extends LJ_F[T, Sequent[G, C]]:
        val ev: Elem[G, i.type] =:= (A => B) = summon

    case class `r=>`[T[_], G <: Tuple: Env, A: Type, B: Type](
        t: T[Sequent[A *: G, B]]
    ) extends LJ_F[T, Sequent[G, A => B]]

    given LJ_HFunctor: HFunctor[LJ_C] with 
        def hfmap[F[_], G[_]](nat: F ~> G): LJ_C[F] ~> LJ_C[G] = 
            [A] => (lf: LJ_F[F, A]) => lf match 
                case p: ax[F, g, a] => ax[G, g, a](p.i)(using p.tpe.env, p.tpe.con, p.ev)
                case p: `l=>`[F, g, a, b, c] => `l=>`[G, g, a, b, c](p.i, nat(p.t1), nat(p.t2))(using p.tpe.env, p.tpe.env(p.i).asInstanceOf[Type.implies[a, b]].t1, p.tpe.env(p.i).asInstanceOf[Type.implies[a, b]].t2, p.tpe.con, p.ev)
                case p: `r=>`[F, g, a, b] => `r=>`[G, g, a, b](nat(p.t))(using p.tpe.env, p.tpe.con.asInstanceOf[Type.implies[a,b]].t1, p.tpe.con.asInstanceOf[Type.implies[a,b]].t2)

    given HTraverse[LJ_C] with 
        def traverse[H[_], I[_]: Applicative, J[_]](f: H ~> (I Compose J)): LJ_C[H] ~> (I Compose LJ_C[J]) = 
            [A] => (t: LJ_F[H, A]) => t match 
                case p: ax[H, g, a] => 
                    ax[J, g, a](p.i)(using p.tpe.env, p.tpe.con, p.ev).pure
                case p: `l=>`[H, g, a, b, c] => 
                    Apply[I].ap2(
                        (`l=>`[J, g, a, b, c].apply(
                            p.i, 
                            _: J[Sequent[g,a]], 
                            _: J[Sequent[b *: g, c]]
                        )(using p.tpe.env, p.tpe.env(p.i).asInstanceOf[Type.implies[a, b]].t1, p.tpe.env(p.i).asInstanceOf[Type.implies[a, b]].t2, p.tpe.con, p.ev)).pure[I])(
                        f(p.t1), f(p.t2))
                case p: `r=>`[H, g, a, b] => Functor[I].map(f(p.t))(`r=>`[J, g, a, b].apply(_)(using p.tpe.env, p.tpe.con.asInstanceOf[Type.implies[a,b]].t1, p.tpe.con.asInstanceOf[Type.implies[a,b]].t2)) 

    given LJ_Lambda[T[_]](using L: ND[T]): (LJ_C[T] ~> T) = 
        [S] => (lj: LJ_F[T, S]) => lj match 
            case axiom: ax[T, g, a] =>
                L.v[g,a](axiom.i)(using axiom.ev)

            case limpl: `l=>`[T, g, a, b, c] =>
                L.substz(L.app(L.v(limpl.i)(using limpl.ev))(limpl.t1))(limpl.t2)

            case rimpl: `r=>`[T, g, a, b] =>
                L.abs(rimpl.t)

    given LJ_Tree: LJ_C[Tree] ~> Tree = 
        [A] => (lj: LJ_F[Tree, A]) => lj match 
            case axiom: ax[Tree, g, a] =>
                Tree(s"ax(${axiom.i}): ${Sequent.show(axiom.tpe)}", Nil)
    
            case limpl: `l=>`[Tree, g, a, b, c] =>
                Tree(s"l=>(${limpl.i}): ${Sequent.show(limpl.tpe)}", List(limpl.t2, limpl.t1))
    
            case rimpl: `r=>`[Tree, g, a, b] =>
                Tree(s"r=>: ${Sequent.show(rimpl.tpe)}", List(rimpl.t))

    given LazyList_LJ_Tree: (LazyList HCompose LJ_C)[Tree] ~> Tree = 
        [A] => (ll: LazyList[LJ_F[Tree, A]]) => 
            Tree("*", ll.map(LJ_Tree[A]).toList)

    given LJ_Print: LJ_C[[t] =>> String] ~> ([t] =>> String) = 
        [A] => (lj: LJ_F[[t] =>> String, A]) => lj match 
            case h: ax[[t] =>> String, g, a] => 
                s"ax(${h.i})"

            case h: `l=>`[[t] =>> String, g, a, b, c] => 
                s"l=>(${h.i},${h.t1}, ${h.t2})"

            case h: `r=>`[[t] =>> String, g, a, b] => 
                s"r=>(${h.t})"

defined type LJ
defined type LJ_C
defined class LJ_F
defined object LJ_F

### Type-driven coalgebra

In [32]:
object TDD: 

    import LJ_F.*
    
    object ax: 
        val coalg: Id ~> (Option Compose LJ_C[Id]) = 
            [A] => (t: A) => t match 
                case s: Sequent[g, b] => 
                    indexOf(s.con)(s.env).map: iof => 
                        LJ_F.ax[Id, g, b](iof.i)(
                            using s.env, s.con, iof.eq.asInstanceOf[Elem[g, iof.i.type] =:= b]
                        ).asInstanceOf[LJ_F[Id, A]]
                case _ => None
    
    object `l=>`: 
        val coalg: Id ~> (LazyList Compose LJ_C[Id]) = 
            [A] => (t: A) => t match 
                case s: Sequent[g, c] =>  
                    indexOfImpl(s.env).map: ioi => 
                        LJ_F.`l=>`[Id, g, ioi.a, ioi.b, c](
                            ioi.i, 
                            Sequent(s.env, Type[ioi.a]), 
                            Sequent(Type[ioi.b] *: s.env, s.con))(
                            using s.env, s.env(ioi.i).asInstanceOf[Type.implies[ioi.a, ioi.b]].t1, s.env(ioi.i).asInstanceOf[Type.implies[ioi.a, ioi.b]].t2, s.con, ioi.eq.asInstanceOf[Elem[g, ioi.i.type] =:= (ioi.a => ioi.b)]
                        ).asInstanceOf[LJ_F[Id, A]]
                case _ => LazyList.empty

    object `r=>`: 
        val coalg: Id ~> (Option Compose LJ_C[Id]) = 
            [A] => (t: A) => t match 
                case Sequent(env: Env[g], Type.implies(ta: Type[a], tb: Type[b])) => 
                    Some(LJ_F.`r=>`[Id, g, a, b](
                        Sequent(ta *: env, tb))(
                        using env, ta, tb
                    ).asInstanceOf[LJ_F[Id, A]])
                case _ => None
    
    given Conversion[Id ~> (Option Compose LJ_C[Id]),
                     Id ~> (LazyList Compose LJ_C[Id])] = c => 
        [A] => (t: Id[A]) => LazyList.from(c(t))

    extension (coalg: Id ~> (LazyList Compose LJ_C[Id]))
        def +(c: Id ~> (LazyList Compose LJ_C[Id])): Id ~> (LazyList Compose LJ_C[Id]) = 
            [A] => (t: A) => coalg(t) ++ c(t)

    val Coalg: Id ~> (LazyList Compose LJ_C[Id]) = 
        ax.coalg +     
        `r=>`.coalg +
        `l=>`.coalg 
                

defined object TDD

### Utils

In [33]:
def searchSpace[A: Type]: HMu[LazyList HCompose LJ_C][Sequent[EmptyTuple, A]] = 
    HMuA.unfold[Id, LazyList HCompose LJ_C](TDD.Coalg)
            .apply(Sequent(EmptyTuple, Type[A]))

defined function searchSpace

In [34]:
def proofs[A: Type]: LazyList[LJ[Sequent[EmptyTuple, A]]] = 
    val ss: HMu[LazyList HCompose LJ_C][Sequent[EmptyTuple, A]] = searchSpace[A]
    HMuA.sequenceF_Until[LazyList, LJ_C](10).apply(ss)

defined function proofs

In [35]:
extension [A: Type](outs: LazyList[LJ[Sequent[EmptyTuple, A]]])
    def and[T[_]](alg: LJ_C[T] ~> T): LazyList[T[Sequent[EmptyTuple, A]]] = 
        outs.map(HMuA.fold(alg)(using summon).apply)

defined extension methods 

In [36]:
def printS[A](ss: HMu[LazyList HCompose LJ_C][A])(depth: Int) = 
    println(HMuA.show[LazyList HCompose LJ_C](depth).apply(ss))

defined function printS

In [37]:
def printP[A](ss: HMu[LJ_C][A])(depth: Int): Unit = 
    println(HMuA.show[LJ_C](depth).apply(ss))

defined function printP

# Examples

In [38]:
trait p
trait q
trait r
trait s
trait t

defined trait p
defined trait q
defined trait r
defined trait s
defined trait t

In [39]:
// p, q, r ⊢ s
Sequent[(p,q,r), s]

res39: Sequent[(p, q, r), s] = Sequent(
  env = (atom(s = "p"), atom(s = "q"), atom(s = "r")),
  con = atom(s = "s")
)

In [39]:
TDD.ax.coalg(Sequent[Char *: Int *: EmptyTuple, Int]

In [40]:
TDD.`l=>`.coalg(Sequent[(Char => Int) *: Char *: EmptyTuple, Int])

res40: LazyList[LJ_C[Sequent[*:[Char => Int, *:[Char, EmptyTuple]], Int]]] = LazyList(
  l=>(
    i = 0,
    t1 = Sequent(
      env = (
        implies(t1 = atom(s = "char"), t2 = atom(s = "int")),
        atom(s = "char")
      ),
      con = atom(s = "char")
    ),
    t2 = Sequent(
      env = (
        atom(s = "int"),
        implies(t1 = atom(s = "char"), t2 = atom(s = "int")),
        atom(s = "char")
      ),
      con = atom(s = "int")
    )
  )
)

In [41]:
printS(searchSpace[Int => Char => Int])(3)

*
   └──r=>:  ⊢ (int => (char => int))
      └──*
         └──r=>: int ⊢ (char => int)
            └──*
               └──ax(1): char, int ⊢ int


In [42]:
printP(proofs[Int => Char => Int].head)(3)

r=>:  ⊢ (int => (char => int))
   └──r=>: int ⊢ (char => int)
      └──ax(1): char, int ⊢ int


In [43]:
printS(searchSpace[(Int => Char => Boolean) => (Int => Char) => (Int => Boolean)])(3)

*
   └──r=>:  ⊢ ((int => (char => boolean)) => ((int => char) => (int => boolean)))
      └──*
         ├──l=>(0): (int => (char => boolean)) ⊢ ((int => char) => (int => boolean))
         |   ├──*
         |   |   └──l=>(0): (int => (char => boolean)) ⊢ int
         |   |      ├──...
         |   |      └──...
         |   └──*
         |      ├──l=>(0): (char => boolean), (int => (char => boolean)) ⊢ ((int => char) => (int => boolean))
         |      |   ├──...
         |      |   └──...
         |      ├──l=>(1): (char => boolean), (int => (char => boolean)) ⊢ ((int => char) => (int => boolean))
         |      |   ├──...
         |      |   └──...
         |      └──r=>: (char => boolean), (int => (char => boolean)) ⊢ ((int => char) => (int => boolean))
         |         └──...
         └──r=>: (int => (char => boolean)) ⊢ ((int => char) => (int => boolean))
            └──*
               ├──l=>(0): (int => char), (int => (char => boolean)) ⊢ (int => boolean)
               |   

In [44]:
printP(proofs[(p => q => r) => (p => q) => (p => r)].head)(10)

r=>:  ⊢ ((p => (q => r)) => ((p => q) => (p => r)))
   └──r=>: (p => (q => r)) ⊢ ((p => q) => (p => r))
      └──r=>: (p => q), (p => (q => r)) ⊢ (p => r)
         └──l=>(1): p, (p => q), (p => (q => r)) ⊢ r
            ├──ax(0): p, (p => q), (p => (q => r)) ⊢ p
            └──l=>(2): q, p, (p => q), (p => (q => r)) ⊢ r
               ├──ax(1): q, p, (p => q), (p => (q => r)) ⊢ p
               └──l=>(3): q, q, p, (p => q), (p => (q => r)) ⊢ r
                  ├──ax(2): q, q, p, (p => q), (p => (q => r)) ⊢ p
                  └──l=>(4): q, q, q, p, (p => q), (p => (q => r)) ⊢ r
                     ├──ax(3): q, q, q, p, (p => q), (p => (q => r)) ⊢ p
                     └──l=>(6): q, q, q, q, p, (p => q), (p => (q => r)) ⊢ r
                        ├──ax(4): q, q, q, q, p, (p => q), (p => (q => r)) ⊢ p
                        └──l=>(0): (q => r), q, q, q, q, p, (p => q), (p => (q => r)) ⊢ r
                           ├──ax(1): (q => r), q, q, q, q, p, (p => q), (p => (q => r)) ⊢ q
   

In [46]:
printS(searchSpace[((q => r) => p => q) => (r => s) => ((p => q) => r) => s])(0)

...
